In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import json
from pathlib import Path
from PIL import Image
import timm
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import cv2
from scipy.ndimage import gaussian_filter
import io

# ============================================================================
# DATASET & AUGMENTATION
# ============================================================================

class DeepfakeDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None):
        self.images = [(p, 0) for p in Path(real_dir).glob('*')] + \
                      [(p, 1) for p in Path(fake_dir).glob('*')]
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        path, label = self.images[idx]
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label


def get_transforms(train=True):
    if train:
        return transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(0.2, 0.2, 0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])


# ============================================================================
# MODEL ARCHITECTURE
# ============================================================================

class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d(1),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.3), nn.Linear(256, 128), nn.ReLU(),
            nn.Dropout(0.3), nn.Linear(128, 2)
        )
        
    def forward(self, x):
        x = self.features(x).view(x.size(0), -1)
        return self.classifier(x)


# ============================================================================
# ADVERSARIAL ATTACKS
# ============================================================================

def fgsm_attack(model, images, labels, eps=0.03):
    was_training = model.training
    model.eval()  # Set to eval mode to avoid batch norm issues
    
    images.requires_grad = True
    loss = F.cross_entropy(model(images), labels)
    model.zero_grad()
    loss.backward()
    perturbed = torch.clamp(images + eps * images.grad.sign(), 0, 1).detach()
    
    if was_training:
        model.train()  # Restore training mode
    
    return perturbed


# ============================================================================
# TRAINING FUNCTIONS
# ============================================================================

def train_epoch(model, loader, optimizer, device, use_adv=False, eps_range=[0.02, 0.05]):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for imgs, labels in tqdm(loader, desc="Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        
        # Mix clean and adversarial examples (process entire batch at once)
        if use_adv and np.random.rand() > 0.3:
            eps = np.random.choice(eps_range)
            imgs = fgsm_attack(model, imgs, labels, eps)
        
        outputs = model(imgs)
        loss = F.cross_entropy(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    
    return total_loss / len(loader), 100. * correct / total


def validate(model, loader, device):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for imgs, labels in loader:
            outputs = model(imgs.to(device))
            probs = F.softmax(outputs, 1)
            all_preds.extend(outputs.argmax(1).cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
    
    acc = accuracy_score(all_labels, all_preds) * 100
    f1 = f1_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    return acc, f1, auc


# ============================================================================
# ADVERSARIAL EVALUATION & VISUALIZATION
# ============================================================================

def eval_adversarial(model, loader, device, epsilons=[0.0, 0.02, 0.05, 0.08]):
    results = {}
    for eps in epsilons:
        all_preds, all_labels = [], []
        for imgs, labels in tqdm(loader, desc=f"ε={eps}"):
            imgs = imgs.to(device)
            if eps > 0:
                imgs = fgsm_attack(model, imgs, labels.to(device), eps)
            with torch.no_grad():
                preds = model(imgs).argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
        
        acc = accuracy_score(all_labels, all_preds) * 100
        results[eps] = acc
        print(f"ε={eps:.3f}: {acc:.2f}%")
    return results


def visualize_adversarial(model, imgs, labels, device, save_path):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    epsilons = [0.0, 0.02, 0.05, 0.08]
    
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    
    for i in range(4):
        img = imgs[i:i+1].to(device)
        label = labels[i].item()
        
        for j, eps in enumerate(epsilons):
            adv_img = fgsm_attack(model, img, labels[i:i+1].to(device), eps) if eps > 0 else img
            
            with torch.no_grad():
                pred = model(adv_img).argmax(1).item()
            
            # Denormalize
            vis_img = (adv_img.cpu()[0] * std + mean).clamp(0, 1)
            vis_img = (vis_img.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
            
            axes[i, j].imshow(vis_img)
            color = 'green' if pred == label else 'red'
            axes[i, j].set_title(f"ε={eps}\n{'✓' if pred==label else '✗'}", 
                                color=color, fontweight='bold')
            axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


def plot_comparison(before, after, save_path):
    eps_list = sorted(before.keys())
    
    plt.figure(figsize=(10, 6))
    plt.plot(eps_list, [before[e] for e in eps_list], 'o-', label='Before', linewidth=2)
    plt.plot(eps_list, [after[e] for e in eps_list], 's-', label='After', linewidth=2)
    plt.xlabel('Epsilon', fontsize=12)
    plt.ylabel('Accuracy (%)', fontsize=12)
    plt.title('Adversarial Robustness Comparison', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


# ============================================================================
# GRAD-CAM
# ============================================================================

class GradCAM:
    def __init__(self, model, layer):
        self.model = model
        self.layer = layer
        self.gradients = None
        self.activations = None
        layer.register_forward_hook(lambda m, i, o: setattr(self, 'activations', o.detach()))
        layer.register_backward_hook(lambda m, gi, go: setattr(self, 'gradients', go[0].detach()))
    
    def generate(self, img, target=None):
        self.model.eval()
        out = self.model(img)
        target = target or out.argmax(1).item()
        self.model.zero_grad()
        out[0, target].backward()
        
        weights = self.gradients[0].mean((1, 2), keepdim=True)
        cam = (weights * self.activations[0]).sum(0)
        cam = F.relu(cam)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        return cam.detach().cpu().numpy()


def visualize_gradcam(model, imgs, labels, device, layer, save_path):
    gradcam = GradCAM(model, layer)
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    fig, axes = plt.subplots(4, 3, figsize=(12, 16))
    
    for i in range(4):
        img = imgs[i:i+1].to(device)
        
        # Original
        vis_img = (imgs[i] * std + mean).clamp(0, 1).permute(1, 2, 0).numpy()
        vis_img = (vis_img * 255).astype(np.uint8)
        axes[i, 0].imshow(vis_img)
        axes[i, 0].set_title('Original')
        axes[i, 0].axis('off')
        
        # CAM
        cam = gradcam.generate(img)
        cam_resized = cv2.resize(cam, (vis_img.shape[1], vis_img.shape[0]))
        heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        axes[i, 1].imshow(heatmap)
        axes[i, 1].set_title('Heatmap')
        axes[i, 1].axis('off')
        
        # Overlay
        overlay = (heatmap * 0.5 + vis_img * 0.5).astype(np.uint8)
        axes[i, 2].imshow(overlay)
        axes[i, 2].set_title('Overlay')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


# ============================================================================
# CORRUPTION TESTING
# ============================================================================

def add_corruption(img, corruption_type, severity=3):
    img_np = np.array(img)
    
    if corruption_type == 'gaussian_noise':
        noise = np.random.normal(0, [0.04, 0.06, 0.08][severity-1], img_np.shape)
        img_np = np.clip(img_np + noise * 255, 0, 255)
    elif corruption_type == 'gaussian_blur':
        sigma = [0.6, 0.8, 1.0][severity-1]
        img_np = gaussian_filter(img_np, sigma=sigma)
    elif corruption_type == 'jpeg':
        quality = [65, 50, 40][severity-1]
        buf = io.BytesIO()
        Image.fromarray(img_np.astype(np.uint8)).save(buf, 'JPEG', quality=quality)
        buf.seek(0)
        return Image.open(buf)
    
    return Image.fromarray(img_np.astype(np.uint8))


def test_corruptions(model, loader, device):
    corruptions = ['gaussian_noise', 'gaussian_blur', 'jpeg']
    results = {'clean': validate(model, loader, device)[0]}
    
    for corruption in corruptions:
        all_preds, all_labels = [], []
        for imgs, labels in tqdm(loader, desc=corruption):
            corrupted = []
            for img in imgs:
                vis = (img.permute(1,2,0).numpy() * [0.229,0.224,0.225] + [0.485,0.456,0.406]) * 255
                vis = add_corruption(Image.fromarray(vis.astype(np.uint8)), corruption, 3)
                corrupted.append(get_transforms(False)(vis))
            
            corrupted = torch.stack(corrupted).to(device)
            with torch.no_grad():
                preds = model(corrupted).argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
        
        results[corruption] = accuracy_score(all_labels, all_preds) * 100
        print(f"{corruption}: {results[corruption]:.2f}%")
    
    return results


# ============================================================================
# MAIN PIPELINE
# ============================================================================

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}\n")
    
    # Data
    train_data = DeepfakeDataset('/kaggle/input/fake-real/data/real_cifake_images', '/kaggle/input/fake-real/data/fake_cifake_images', get_transforms(True))
    val_data = DeepfakeDataset('/kaggle/input/fake-real/data/real_cifake_images', '/kaggle/input/fake-real/data/fake_cifake_images', get_transforms(False))
    train_loader = DataLoader(train_data, 64, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_data, 64, num_workers=4, pin_memory=True)
    
    # Models
    models = {
        'efficientnet': timm.create_model('efficientnet_b0', pretrained=True, num_classes=2),
        'custom_cnn': CustomCNN()
    }
    
    trained_models = []
    
    for name, model in models.items():
        print(f"\n{'='*60}\nTraining: {name}\n{'='*60}")
        model = model.to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
        
        # Phase 1: Normal training
        best_auc = 0
        for epoch in range(50):
            train_loss, train_acc = train_epoch(model, train_loader, optimizer, device)
            val_acc, val_f1, val_auc = validate(model, val_loader, device)
            print(f"Epoch {epoch+1}: Train {train_acc:.2f}% | Val {val_acc:.2f}% F1={val_f1:.3f} AUC={val_auc:.3f}")
            
            if val_auc > best_auc:
                best_auc = val_auc
                torch.save(model.state_dict(), f'{name}_base.pth')
        
        model.load_state_dict(torch.load(f'{name}_base.pth'))
        
        # Adversarial evaluation (before)
        print(f"\n--- Adversarial Testing (Before) ---")
        sample_imgs, sample_labels = next(iter(val_loader))
        visualize_adversarial(model, sample_imgs[:4], sample_labels[:4], device, 
                            f'{name}_adv_before.png')
        before_results = eval_adversarial(model, val_loader, device)
        
        # Phase 2: Adversarial fine-tuning
        print(f"\n--- Adversarial Fine-tuning ---")
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
        for epoch in range(25):
            loss, acc = train_epoch(model, train_loader, optimizer, device, use_adv=True)
            val_acc, val_f1, val_auc = validate(model, val_loader, device)
            print(f"Epoch {epoch+1}: Train {acc:.2f}% | Val {val_acc:.2f}% F1={val_f1:.3f}")
        
        torch.save(model.state_dict(), f'{name}_final.pth')
        
        # Adversarial evaluation (after)
        print(f"\n--- Adversarial Testing (After) ---")
        visualize_adversarial(model, sample_imgs[:4], sample_labels[:4], device, 
                            f'{name}_adv_after.png')
        after_results = eval_adversarial(model, val_loader, device)
        plot_comparison(before_results, after_results, f'{name}_comparison.png')
        
        # Grad-CAM
        layer = model.conv_head if 'efficient' in name else model.features[-2]
        visualize_gradcam(model, sample_imgs[:4], sample_labels[:4], device, layer, 
                         f'{name}_gradcam.png')
        
        # Corruption testing
        print(f"\n--- Corruption Testing ---")
        corruption_results = test_corruptions(model, val_loader, device)
        
        trained_models.append(model)
    
    # Ensemble predictions
    print(f"\n{'='*60}\nGenerating Predictions\n{'='*60}")
    test_dir = Path('/kaggle/input/fake-real/data/test')
    predictions = []
    transform = get_transforms(False)
    
    for idx, path in enumerate(sorted(test_dir.glob('*'))):
        img = transform(Image.open(path).convert('RGB')).unsqueeze(0).to(device)
        
        with torch.no_grad():
            probs = sum(F.softmax(m(img), 1) for m in trained_models) / len(trained_models)
        
        pred = "fake" if probs[0, 1] > 0.5 else "real"
        predictions.append({"index": idx + 1, "prediction": pred})
    
    with open('submission.json', 'w') as f:
        json.dump(predictions, f, indent=4)
    
    print(f"\n✓ Predictions saved: {len(predictions)} images")


if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Device: cuda



model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]


Training: efficientnet


Training: 100%|██████████| 32/32 [00:04<00:00,  6.61it/s]


...

gaussian_noise: 84.55%


gaussian_blur: 100%|██████████| 32/32 [00:01<00:00, 18.08it/s]


gaussian_blur: 57.55%


jpeg: 100%|██████████| 32/32 [00:01<00:00, 18.02it/s]


jpeg: 90.35%

Generating Predictions

✓ Predictions saved: 500 images
